In [ ]:
import numpy as np 
import json 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from scipy import sparse 
import pickle 

In [ ]:
train_path ='./data/train_processed.csv'
val_path =  './data/val_processed.csv'
test_path = './data/test_processed.csv'

train = pd.read_csv(train_path)
val = pd.read_csv(val_path)
test = pd.read_csv(test_path)

In [ ]:
train.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,review_id,user_id,business_id,stars,date,text,useful,...,year,month,day,language,cleaned_text,funny_label,cool_label,useful_label,funny_sampled_flag,cool_sampled_flag
0,0,0,0,O4RZMP8IFyJTNfRp0QXEsw,u0LXt3Uea_GidxRW1xcsfg,vcxvQyAggPqxcHwvJXvjGg,4,2017-01-04,Love this place!\n\nThe cakes are delicious bu...,0.0,...,2017.0,1.0,4.0,en,Love place cake delicious really rich cute cup...,0,0,0,0,0
1,1,1,1,66KqTwiQ1oB9-aTsoEN35Q,u0LXt3Uea_GidxRW1xcsfg,DKiRDPtQ5cTN-eX1oEgA9w,3,2017-01-04,It's a pub... nice and clean one.\n\nCame here...,0.0,...,2017.0,1.0,4.0,en,pub nice clean one Came grab food girl since T...,0,0,0,0,0
2,2,2,2,3KQ3_xutS5R3mX7HsPWpgw,u0LXt3Uea_GidxRW1xcsfg,zkU-WMio8g6dpRJ2Y2xqvQ,4,2017-01-04,Came here for dim sum - it's one of those mark...,0.0,...,2017.0,1.0,4.0,en,Came dim sum one mark sheet order dim sum pret...,0,0,0,0,0
3,3,3,3,p659sFGjbL8chCRZPPyFmg,u0LXt3Uea_GidxRW1xcsfg,3RlylOY452bA8rwliPUeUQ,4,2017-01-04,My sister loves this place! Probably comes by ...,1.0,...,2017.0,1.0,4.0,en,sister love place Probably come week every two...,0,0,1,1,1
4,4,4,4,1XEm85LhDeiUHdG0R_rXSQ,u0LXt3Uea_GidxRW1xcsfg,sTV4qyjDkY5cLvx2omtLyw,4,2017-02-07,Nice addition to the Distillery district! Stum...,1.0,...,2017.0,2.0,7.0,en,Nice addition Distillery district Stumbled upo...,0,0,1,0,0


In [ ]:
train['l1']=train.text.str.split().apply(len)
train['l2']=train.cleaned_text.str.split().apply(len)
train['l3']=train.text.apply(len)
train['l4']=train.cleaned_text.apply(len)

In [ ]:
# train_use = train
train_fun = train[train.funny_sampled_flag==1]
train_cool = train[train.cool_sampled_flag==1]

In [ ]:
val['l1']=val.text.str.split().apply(len)
val['l2']=val.cleaned_text.str.split().apply(len)
val['l3']=val.text.apply(len)
val['l4']=val.cleaned_text.apply(len)

test['l1']=test.text.str.split().apply(len)
test['l2']=test.cleaned_text.str.split().apply(len)
test['l3']=test.text.apply(len)
test['l4']=test.cleaned_text.apply(len)

In [ ]:
for i in [train, train_fun, train_cool, val, test]:
    i.dropna(inplace=True)
    print(i.shape)

(896945, 26)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(294305, 26)
(376157, 26)
(98119, 23)
(126864, 23)


In [ ]:
train.reset_index(inplace=True)
train_fun.reset_index(inplace=True)
train_cool.reset_index(inplace=True)
val.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
from sklearn import tree, ensemble
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, mean_squared_error
from xgboost import XGBClassifier

# Useful

In [ ]:
model = 'doc2vec'
with open(f'./text_processing/useful_processed_text/useful_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/useful_processed_text/useful_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/useful_processed_text/useful_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)

for i in [train_array, val_array, test_array]:
    print(i.shape)

(896945, 60)
(98119, 60)
(126864, 60)


In [ ]:
train_array_new = np.zeros((train_array.shape[0],train_array.shape[1]+4))
train_array_new[:,:-4] = train_array
train_array_new[:,-4] = np.array(train.l1.tolist())
train_array_new[:,-3] = np.array(train.l2.tolist())
train_array_new[:,-2] = np.array(train.l3.tolist())
train_array_new[:,-1] = np.array(train.l4.tolist())
train_array_new.shape

(896945, 64)

In [ ]:
val_array_new = np.zeros((val_array.shape[0],val_array.shape[1]+4))
val_array_new[:,:-4] = val_array
val_array_new[:,-4] = np.array(val.l1.tolist())
val_array_new[:,-3] = np.array(val.l2.tolist())
val_array_new[:,-2] = np.array(val.l3.tolist())
val_array_new[:,-1] = np.array(val.l4.tolist())
val_array_new.shape

(98119, 64)

In [ ]:
test_array_new = np.zeros((test_array.shape[0],test_array.shape[1]+4))
test_array_new[:,:-4] = test_array
test_array_new[:,-4] = np.array(test.l1.tolist())
test_array_new[:,-3] = np.array(test.l2.tolist())
test_array_new[:,-2] = np.array(test.l3.tolist())
test_array_new[:,-1] = np.array(test.l4.tolist())
test_array_new.shape

(126864, 64)

In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  1.0
confusion matrix: 
[[594845      0]
 [     0 302100]]
accuracy score:  0.6065899570929177
confusion matrix: 
[[49394 24207]
 [14394 10124]]
accuracy score:  0.614792218438643
confusion matrix: 
[[66360 33670]
 [15199 11635]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=10)
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6970906800305482
confusion matrix: 
[[538317  56528]
 [215165  86935]]
accuracy score:  0.7369214932887616
confusion matrix: 
[[65086  8515]
 [17298  7220]]
accuracy score:  0.7556990162693908
confusion matrix: 
[[87296 12734]
 [18259  8575]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(min_samples_split=130)
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.761637558601698
confusion matrix: 
[[523045  71800]
 [141998 160102]]
accuracy score:  0.6684128456262294
confusion matrix: 
[[56503 17098]
 [15437  9081]]
accuracy score:  0.6784982343296758
confusion matrix: 
[[75721 24309]
 [16478 10356]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=10,min_samples_split=130)
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6963938703041993
confusion matrix: 
[[538026  56819]
 [215499  86601]]
accuracy score:  0.7368705347588133
confusion matrix: 
[[65084  8517]
 [17301  7217]]
accuracy score:  0.7555177197628957
confusion matrix: 
[[87279 12751]
 [18265  8569]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=200)
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.689958693119422
confusion matrix: 
[[541365  53480]
 [224610  77490]]
accuracy score:  0.7431486256484473
confusion matrix: 
[[66062  7539]
 [17663  6855]]
accuracy score:  0.7633134695421869
confusion matrix: 
[[88737 11293]
 [18734  8100]]


In [ ]:
dt_estimator = ensemble.GradientBoostingClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6928496173120983
confusion matrix: 
[[541191  53654]
 [221843  80257]]
accuracy score:  0.7445143142510625
confusion matrix: 
[[65913  7688]
 [17380  7138]]
accuracy score:  0.7619813343422878
confusion matrix: 
[[88294 11736]
 [18460  8374]]


In [ ]:
dt_estimator = ensemble.AdaBoostClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6869384410415355
confusion matrix: 
[[551419  43426]
 [237373  64727]]
accuracy score:  0.7462265208573263
confusion matrix: 
[[67555  6046]
 [18854  5664]]
accuracy score:  0.7688863665027116
confusion matrix: 
[[90918  9112]
 [20208  6626]]


In [ ]:
dt_estimator = XGBClassifier()
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6928039065940498
confusion matrix: 
[[541467  53378]
 [222160  79940]]
accuracy score:  0.7444022054851762
confusion matrix: 
[[65916  7685]
 [17394  7124]]
accuracy score:  0.7624306343801236
confusion matrix: 
[[88341 11689]
 [18450  8384]]


# Funny

In [ ]:
model = 'doc2vec'
with open(f'./text_processing/funny_processed_text/funny_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/funny_processed_text/funny_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/funny_processed_text/funny_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)

for i in [train_array_fun, val_array_fun, test_array_fun]:
    print(i.shape)

(294305, 60)
(98119, 60)
(126864, 60)


In [ ]:
train_array_fun_new = np.zeros((train_array_fun.shape[0],train_array_fun.shape[1]+4))
train_array_fun_new[:,:-4] = train_array_fun
train_array_fun_new[:,-4] = np.array(train_fun.l1.tolist())
train_array_fun_new[:,-3] = np.array(train_fun.l2.tolist())
train_array_fun_new[:,-2] = np.array(train_fun.l3.tolist())
train_array_fun_new[:,-1] = np.array(train_fun.l4.tolist())
train_array_fun_new.shape

(294305, 64)

In [ ]:
val_array_fun_new = np.zeros((val_array_fun.shape[0],val_array_fun.shape[1]+4))
val_array_fun_new[:,:-4] = val_array_fun
val_array_fun_new[:,-4] = np.array(val.l1.tolist())
val_array_fun_new[:,-3] = np.array(val.l2.tolist())
val_array_fun_new[:,-2] = np.array(val.l3.tolist())
val_array_fun_new[:,-1] = np.array(val.l4.tolist())
val_array_fun_new.shape

(98119, 64)

In [ ]:
test_array_fun_new = np.zeros((test_array_fun.shape[0],test_array_fun.shape[1]+4))
test_array_fun_new[:,:-4] = test_array_fun
test_array_fun_new[:,-4] = np.array(test.l1.tolist())
test_array_fun_new[:,-3] = np.array(test.l2.tolist())
test_array_fun_new[:,-2] = np.array(test.l3.tolist())
test_array_fun_new[:,-1] = np.array(test.l4.tolist())
test_array_fun_new.shape

(126864, 64)

In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  1.0
confusion matrix: 
[[176583      0]
 [     0 117722]]
accuracy score:  0.6132043742802108
confusion matrix: 
[[55952 33204]
 [ 4748  4215]]
accuracy score:  0.614161621894312
confusion matrix: 
[[72803 43633]
 [ 5316  5112]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=10)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6648952617182855
confusion matrix: 
[[143128  33455]
 [ 65168  52554]]
accuracy score:  0.7574373974459585
confusion matrix: 
[[70289 18867]
 [ 4933  4030]]
accuracy score:  0.7560616092823811
confusion matrix: 
[[90942 25494]
 [ 5453  4975]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=15)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.704323745773942
confusion matrix: 
[[149885  26698]
 [ 60321  57401]]
accuracy score:  0.7541964349412448
confusion matrix: 
[[70176 18980]
 [ 5138  3825]]
accuracy score:  0.7525696809181486
confusion matrix: 
[[90719 25717]
 [ 5673  4755]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=50)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6553745264266662
confusion matrix: 
[[143236  33347]
 [ 68078  49644]]
accuracy score:  0.7695451441616812
confusion matrix: 
[[71492 17664]
 [ 4948  4015]]
accuracy score:  0.7668605751040485
confusion matrix: 
[[92308 24128]
 [ 5449  4979]]


In [ ]:
dt_estimator = ensemble.GradientBoostingClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6549837753351115
confusion matrix: 
[[143783  32800]
 [ 68740  48982]]
accuracy score:  0.7729593656682192
confusion matrix: 
[[71860 17296]
 [ 4981  3982]]
accuracy score:  0.7706599192836423
confusion matrix: 
[[92839 23597]
 [ 5498  4930]]


In [ ]:
dt_estimator = ensemble.AdaBoostClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6538013285537113
confusion matrix: 
[[143594  32989]
 [ 68899  48823]]
accuracy score:  0.7727453398424362
confusion matrix: 
[[71839 17317]
 [ 4981  3982]]
accuracy score:  0.7702657964434355
confusion matrix: 
[[92799 23637]
 [ 5508  4920]]


In [ ]:
dt_estimator = XGBClassifier()
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6546711744618678
confusion matrix: 
[[143768  32815]
 [ 68817  48905]]
accuracy score:  0.7732141583179608
confusion matrix: 
[[71887 17269]
 [ 4983  3980]]
accuracy score:  0.770754508765292
confusion matrix: 
[[92859 23577]
 [ 5506  4922]]


In [ ]:
model = 'sbert'
with open(f'./text_processing/funny_processed_text/funny_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/funny_processed_text/funny_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/funny_processed_text/funny_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)

for i in [train_array_fun, val_array_fun, test_array_fun]:
    print(i.shape)

(294305, 768)
(98119, 768)
(126864, 768)


In [ ]:
train_array_fun_new = np.zeros((train_array_fun.shape[0],train_array_fun.shape[1]+4))
train_array_fun_new[:,:-4] = train_array_fun
train_array_fun_new[:,-4] = np.array(train_fun.l1.tolist())
train_array_fun_new[:,-3] = np.array(train_fun.l2.tolist())
train_array_fun_new[:,-2] = np.array(train_fun.l3.tolist())
train_array_fun_new[:,-1] = np.array(train_fun.l4.tolist())
train_array_fun_new.shape

(294305, 772)

In [ ]:
val_array_fun_new = np.zeros((val_array_fun.shape[0],val_array_fun.shape[1]+4))
val_array_fun_new[:,:-4] = val_array_fun
val_array_fun_new[:,-4] = np.array(val.l1.tolist())
val_array_fun_new[:,-3] = np.array(val.l2.tolist())
val_array_fun_new[:,-2] = np.array(val.l3.tolist())
val_array_fun_new[:,-1] = np.array(val.l4.tolist())
val_array_fun_new.shape

(98119, 772)

In [ ]:
test_array_fun_new = np.zeros((test_array_fun.shape[0],test_array_fun.shape[1]+4))
test_array_fun_new[:,:-4] = test_array_fun
test_array_fun_new[:,-4] = np.array(test.l1.tolist())
test_array_fun_new[:,-3] = np.array(test.l2.tolist())
test_array_fun_new[:,-2] = np.array(test.l3.tolist())
test_array_fun_new[:,-1] = np.array(test.l4.tolist())
test_array_fun_new.shape

(126864, 772)

In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  1.0
confusion matrix: 
[[176583      0]
 [     0 117722]]
accuracy score:  0.5956440648600169
confusion matrix: 
[[54191 34965]
 [ 4710  4253]]
accuracy score:  0.593927355278093
confusion matrix: 
[[70292 46144]
 [ 5372  5056]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=30)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.9264810315828818
confusion matrix: 
[[172805   3778]
 [ 17859  99863]]
accuracy score:  0.6513111629755705
confusion matrix: 
[[59897 29259]
 [ 4954  4009]]
accuracy score:  0.650483982847774
confusion matrix: 
[[77675 38761]
 [ 5580  4848]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=15)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.717327262533766
confusion matrix: 
[[148757  27826]
 [ 55366  62356]]
accuracy score:  0.7302459258655306
confusion matrix: 
[[67581 21575]
 [ 4893  4070]]
accuracy score:  0.7287646613696557
confusion matrix: 
[[87371 29065]
 [ 5345  5083]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7,min_samples_split=50)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6565705645503814
confusion matrix: 
[[146076  30507]
 [ 70566  47156]]
accuracy score:  0.7721236457770666
confusion matrix: 
[[71812 17344]
 [ 5015  3948]]
accuracy score:  0.769375078824568
confusion matrix: 
[[92773 23663]
 [ 5595  4833]]


# Cool

In [ ]:
model = 'doc2vec'
with open(f'./text_processing/cool_processed_text/cool_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/cool_processed_text/cool_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/cool_processed_text/cool_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)
    
for i in [train_array_cool, val_array_cool, test_array_cool]:
    print(i.shape)

(376157, 60)
(98119, 60)
(126864, 60)


In [ ]:
train_array_cool_new = np.zeros((train_array_cool.shape[0],train_array_cool.shape[1]+4))
train_array_cool_new[:,:-4] = train_array_cool
train_array_cool_new[:,-4] = np.array(train_cool.l1.tolist())
train_array_cool_new[:,-3] = np.array(train_cool.l2.tolist())
train_array_cool_new[:,-2] = np.array(train_cool.l3.tolist())
train_array_cool_new[:,-1] = np.array(train_cool.l4.tolist())
train_array_cool_new.shape

(376157, 64)

In [ ]:
val_array_cool_new = np.zeros((val_array_cool.shape[0],val_array_cool.shape[1]+4))
val_array_cool_new[:,:-4] = val_array_cool
val_array_cool_new[:,-4] = np.array(val.l1.tolist())
val_array_cool_new[:,-3] = np.array(val.l2.tolist())
val_array_cool_new[:,-2] = np.array(val.l3.tolist())
val_array_cool_new[:,-1] = np.array(val.l4.tolist())
val_array_cool_new.shape

(98119, 64)

In [ ]:
test_array_cool_new = np.zeros((test_array_cool.shape[0],test_array_cool.shape[1]+4))
test_array_cool_new[:,:-4] = test_array_cool
test_array_cool_new[:,-4] = np.array(test.l1.tolist())
test_array_cool_new[:,-3] = np.array(test.l2.tolist())
test_array_cool_new[:,-2] = np.array(test.l3.tolist())
test_array_cool_new[:,-1] = np.array(test.l4.tolist())
test_array_cool_new.shape

(126864, 64)

In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  1.0
confusion matrix: 
[[225694      0]
 [     0 150463]]
accuracy score:  0.5927700037709313
confusion matrix: 
[[51849 32450]
 [ 7507  6313]]
accuracy score:  0.5965285660234582
confusion matrix: 
[[67864 42531]
 [ 8655  7814]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=20)
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.7647976775654847
confusion matrix: 
[[205967  19727]
 [ 68746  81717]]
accuracy score:  0.7155902526523915
confusion matrix: 
[[65162 19137]
 [ 8769  5051]]
accuracy score:  0.7212999747761383
confusion matrix: 
[[84960 25435]
 [ 9922  6547]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=30)
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.9131905029017139
confusion matrix: 
[[220951   4743]
 [ 27911 122552]]
accuracy score:  0.6548374932479948
confusion matrix: 
[[58566 25733]
 [ 8134  5686]]
accuracy score:  0.6570027746247951
confusion matrix: 
[[76228 34167]
 [ 9347  7122]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=50)
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6383903529643208
confusion matrix: 
[[196903  28791]
 [107231  43232]]
accuracy score:  0.7868506609321334
confusion matrix: 
[[73068 11231]
 [ 9683  4137]]
accuracy score:  0.7937555177197629
confusion matrix: 
[[95250 15145]
 [11020  5449]]


In [ ]:
dt_estimator = ensemble.GradientBoostingClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6378453677586753
confusion matrix: 
[[190207  35487]
 [100740  49723]]
accuracy score:  0.7681590721470867
confusion matrix: 
[[70610 13689]
 [ 9059  4761]]
accuracy score:  0.7733478370538529
confusion matrix: 
[[91889 18506]
 [10248  6221]]


In [ ]:
dt_estimator = ensemble.AdaBoostClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6357478393330445
confusion matrix: 
[[190107  35587]
 [101429  49034]]
accuracy score:  0.7678431292614071
confusion matrix: 
[[70639 13660]
 [ 9119  4701]]
accuracy score:  0.774041493252617
confusion matrix: 
[[92005 18390]
 [10276  6193]]


In [ ]:
dt_estimator = XGBClassifier()
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6378054907924084
confusion matrix: 
[[190851  34843]
 [101399  49064]]
accuracy score:  0.7696062943976192
confusion matrix: 
[[70803 13496]
 [ 9110  4710]]
accuracy score:  0.7751450372051961
confusion matrix: 
[[92174 18221]
 [10305  6164]]
